## First MongoDB Companies Filter

In [6]:
from pymongo import MongoClient
import pandas as pd
import re
import getpass
import json
import os
import requests
from dotenv import load_dotenv
load_dotenv()
import numpy as np
import functions as func

- Connecting with mongoclient to read database:

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

- Reading companies database:

In [3]:
db, coll = connectCollection('companies','companies')

- Filtering by: no deadpooled year and at least one office:

In [4]:
companies = list(coll.find({"$and": [{"deadpooled_year":None},{"$where":"this.offices.length >= 1"}]}))
#company2 = list(coll.find({"$and": [{"deadpooled_year":None},{"$where":"this.offices.length >= 1"},{"$or":[{"$where":"this.funding_rounds.length >= 1"},{"total_money_raised":{"$exists":True}}]}]}))


In [5]:
len(companies)

12957

- Creating a list of dictionaries with the information of each company and the latitude and longitude of each office of each company:

In [6]:
merged_offices = []

for c in companies:
    offices = c.get('offices',[])
    for office in offices:
        merged_office = dict()
        raised_amount=0
        merged_office["ref"] = str(c.get('_id',0))
        merged_office["Name"]=c.get('name',0)
        merged_office["Category"]=c.get('category_code',0)
        merged_office["Description"]=c.get('description',0)
        merged_office["Latitude"]=office.get('latitude',0)
        merged_office["Longitude"]=office.get('longitude',0)
        merged_office["Founded Year"]=c.get('founded_year',0)
        merged_office["Total Money Raised"]=c.get('total_money_raised',0)
        for rounds in range(len(c.get('funding_rounds'))):
            r=c.get('funding_rounds')[rounds].get('raised_amount',0)
            if r !=None:
                raised_amount+=r
        merged_office["Raised Amount"]=raised_amount
        if len(c.get('funding_rounds'))>0:
            merged_office["Currency"]=c.get('funding_rounds')[0].get('raised_currency_code',0)
        else:
            merged_office["Currency"]=0
        merged_offices.append(merged_office)
        merged_office["Address"]=office.get('address1',0)
        merged_office["Zip code"]=office.get('zip_code',0)
        merged_office["City"]=office.get('city',0)
        


- Creating dataframe with the information:

In [7]:
df=pd.DataFrame(merged_offices)
df.head()

,ref,Name,Category,Description,Latitude,Longitude,Founded Year,Total Money Raised,Raised Amount,Currency,Address,Zip code,City
0,52cdef7c4bab8bd675297d8d,Digg,news,user driven social content website,37.764726,-122.394523,2004.0,$45M,45000000.0,USD,135 Mississippi St,94107,San Francisco
1,52cdef7c4bab8bd675297d90,Postini,web,None,37.506885,-122.247573,1999.0,$0,0.0,0,"959 Skyway Road, Suite 200",94070,San Carlos
2,52cdef7c4bab8bd675297d92,Flektor,games_video,None,34.025958,-118.379768,NaN,$0,0.0,0,"8536 National Blvd, Suite A",90232,Culver City
3,52cdef7c4bab8bd675297d91,Geni,web,Geneology social network site,34.090368,-118.393064,2006.0,$16.5M,16500000.0,USD,9229 W. Sunset Blvd.,90069,West Hollywood
4,52cdef7c4bab8bd675297d93,Fox Interactive Media,web,,34.076179,-118.394170,1979.0,$0,0.0,0,407 N Maple Dr,90210,Beverly Hills


In [8]:
print(set(df['Currency']))
df.shape

{0, 'JPY', 'USD', 'EUR', 'SEK', None, 'GBP', 'CAD'}


(15848, 13)

- Adding currency to all the rows in the dataframe:

In [10]:
money={'$': 'USD', 'C$': 'CAD', '€': 'EUR', '£': 'GBP', 'kr': 'SEK', '¥': 'JPY'}
for key,value in money.items():
    df.loc[(df['Total Money Raised'].str.startswith(key)),'Currency']=value
df.head()

,ref,Name,Category,Description,Latitude,Longitude,Founded Year,Total Money Raised,Raised Amount,Currency,Address,Zip code,City
0,52cdef7c4bab8bd675297d8d,Digg,news,user driven social content website,37.764726,-122.394523,2004.0,$45M,45000000.0,USD,135 Mississippi St,94107,San Francisco
1,52cdef7c4bab8bd675297d90,Postini,web,None,37.506885,-122.247573,1999.0,$0,0.0,USD,"959 Skyway Road, Suite 200",94070,San Carlos
2,52cdef7c4bab8bd675297d92,Flektor,games_video,None,34.025958,-118.379768,NaN,$0,0.0,USD,"8536 National Blvd, Suite A",90232,Culver City
3,52cdef7c4bab8bd675297d91,Geni,web,Geneology social network site,34.090368,-118.393064,2006.0,$16.5M,16500000.0,USD,9229 W. Sunset Blvd.,90069,West Hollywood
4,52cdef7c4bab8bd675297d93,Fox Interactive Media,web,,34.076179,-118.394170,1979.0,$0,0.0,USD,407 N Maple Dr,90210,Beverly Hills


In [11]:
set(df['Currency'])

{'CAD', 'EUR', 'GBP', 'JPY', 'SEK', 'USD'}

- Creating geolocation dictionary with correct format

In [5]:
def getLocation(long,lat):
    loc = {
        'type':'Point',
        'coordinates':[float(long), float(lat)]
    }
    return loc

- Requesting the Google API for the empty longitudes and latitudes in the dataframe:

In [17]:
def googleRequestAuthorized(address):
    authToken = os.getenv("API_KEY")
    if not authToken:
        raise ValueError("NECESITAS UN TOKEN")
    else:
        url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={authToken}"
        res = requests.get(url)

    return res

In [18]:
def apicompanies(ad):
    loc_comp = googleRequestAuthorized(ad).json()
    return loc_comp

- Dataframe with the filter of money raised>1M USD having in mind that there are different coins.

In [15]:
df_1M=df[(((df['Raised Amount']>=1000000) & (df['Currency']=="USD"))|((df['Raised Amount']>=1320000) & (df['Currency']=="CAD"))|((df['Raised Amount']>=900000) & (df['Currency']=="EUR"))|((df['Raised Amount']>=770000) & (df['Currency']=="GBP"))|((df['Raised Amount']>=108560000) & (df['Currency']=="JPY"))|((df['Raised Amount']>=9640000) & (df['Currency']=="SEK")))]
df_1M.shape

(4824, 13)

In [23]:
for _,row in df_1M.iterrows():
    if np.isnan(row['Latitude']):
        if (row['Address']):
            ad=row['Address']
            loc=apicompanies(ad)
            if loc['status']!='ZERO_RESULTS' and loc['status']!='REQUEST_DENIED'and loc['status']!='INVALID_REQUEST':
                df_1M.loc[(df_1M['Address']==ad),'Longitude']=loc['results'][0]['geometry']['location']['lng']
                df_1M.loc[(df_1M['Address']==ad),'Latitude']=loc['results'][0]['geometry']['location']['lat']


In [25]:
df_1M[(df_1M['Longitude'].isnull() & df_1M['Latitude'].isnull())].shape

(325, 13)

- Removing rows with no latitude or longitude:

In [36]:
df_1M_notnull=df_1M[(df_1M['Longitude'].notnull() & df_1M['Latitude'].notnull())]

In [37]:
df_1M_notnull.shape

(4499, 14)

- Adding geolocation column to the dataframe

In [38]:
geolocations=[]
for long, lat in df_1M_notnull[["Longitude","Latitude"]].values:
    geolocations.append(func.getLocation(long,lat))
df_1M_notnull['GeoLocations']=geolocations
df_1M_notnull.shape

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(4499, 14)

- Importing dataframe filtered with 1M, lat, long and geolocations

In [42]:
df_1M_notnull.to_json('./output/1M_company_officess.json', orient="records")
df_1M_notnull.shape

(4499, 14)

- Dataframe with the filter of founded year>=2009 and money raised>1M$

In [40]:
df_FY_1M=df_1M_notnull[(df_1M_notnull['Founded Year']>=2009)]
df_FY_1M.shape

(141, 14)

- Importing dataframe filtered with 1M, lat, long, geolocations and founded year>=2009

In [41]:
#mongoimport --db companies --drop --collection 1MCompanies --jsonArray 1M.json
df_FY_1M.to_json('./output/FY_1M.json', orient="records")